In [ ]:
#!/usr/bin/env python3
# encoding: utf-8
#
# Copyright (C) 2024 Max Planck Institute for Multidisclplinary Sciences
# Copyright (C) 2024 Bharti Arora <bharti.arora@mpinat.mpg.de>
# Copyright (C) 2024 Ajinkya Kulkarni <ajinkya.kulkarni@mpinat.mpg.de>
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU Affero General Public License as
# published by the Free Software Foundation, either version 3 of the
# License, or (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU Affero General Public License for more details.
#
# You should have received a copy of the GNU Affero General Public License
# along with this program. If not, see <https://www.gnu.org/licenses/>.

########################################################################################

In [ ]:
import os
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
from tifffile import imread, imwrite
from stardist import export_imagej_rois, random_label_cmap
from stardist.models import StarDist2D
from stardist.utils import fill_label_holes
import pandas as pd
import cv2
from contextlib import redirect_stdout
from skimage.measure import regionprops_table

import sys
# Don't generate the __pycache__ folder locally
sys.dont_write_bytecode = True 
# Print exception without the buit-in python warning
sys.tracebacklimit = 0

from modules_histo import *

In [ ]:
# Directories
input_folder = '.\input_data'
output_folder = '.\output_data'

########################################################################################

# Ensure the output subfolders exist
os.makedirs(os.path.join(output_folder, 'labeled'), exist_ok=True)
os.makedirs(os.path.join(output_folder, 'density'), exist_ok=True)
os.makedirs(os.path.join(output_folder, 'circularity'), exist_ok=True)
os.makedirs(os.path.join(output_folder, 'results'), exist_ok=True)
os.makedirs(os.path.join(output_folder, 'packaging'), exist_ok=True)
os.makedirs(os.path.join(output_folder, 'area'), exist_ok=True)

########################################################################################

model = StarDist2D.from_pretrained('2D_versatile_he')

########################################################################################

all_images_results = pd.DataFrame(columns=['Filename', 'Percentage_Area_high_nuclei', 
                                           'Percentage_Area_low_nuclei'])

filenames = sorted(os.listdir(input_folder))

In [ ]:
# Process all images in the input folder
for filename in filenames:
    
    if filename.endswith('.tif'):

        # Read image
        img = imread(os.path.join(input_folder, filename))
        
        # Use percentiles 25.0 and 85.0 for normalization
        mi, ma = np.percentile(img, [25.0, 85.0])
        normalizer = MyNormalizer(mi, ma)

        # Optimize the probability and NMS thresholds
        model.config.prob_thresh = 0.4
        model.config.nms_thresh = 0.3

        print(f"Set prob_thresh: {model.config.prob_thresh}")
        print(f"Set nms_thresh: {model.config.nms_thresh}")

        with redirect_stdout(open(os.devnull, "w")) as f:
            # Predict instances and fill label holes
            labels, polys = model.predict_instances_big(img, axes='YXC', block_size=1000,
                                                        min_overlap=144, context=128,
                                                        normalizer=normalizer, n_tiles=(2,2,1),
                                                        show_progress = False)

        labels = fill_label_holes(labels)

        # Save labeled image
        imwrite(os.path.join(output_folder, 'labeled', filename), labels.astype(np.uint16))

        # Calculate and save nuclear density
        Local_Density_mean_filter = mean_filter(labels)
        
        Local_Density_mean_filter = normalize_density_maps(Local_Density_mean_filter)
        imwrite(os.path.join(output_folder, 'density', filename.replace('.tif', '_density.tif')),
               Local_Density_mean_filter.astype(np.float32))

        #Convert to gray and normalize
        image_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        image_gray = normalize_array(image_gray)

        ThresholdValueKey = 200
        
        # Create a mask for accepted pixels.
        # Count the number of pixels in the grayscale image that are above the threshold value.
        accepted_mask = image_gray < ThresholdValueKey

        # Applying the mask to the labels to nullify labels in non-accepted areas.
        # Create a masked array where labels in rejected areas are set to 0.
        labels_masked = labels * accepted_mask

        # Since labels_masked will have labels set to 0 in non-accepted areas,
        # count unique labels excluding 0 (background).
        unique_labels = np.unique(labels_masked)
        num_unique_labels = len(unique_labels) - 1 if 0 in unique_labels else len(unique_labels)
        
        # Obtain the Local_Density_mean_filter
        density_high_nuclei = Local_Density_mean_filter >= 0.5
        density_low_nuclei = Local_Density_mean_filter < 0.5
        
        
        # Apply the accepted_mask to density maps      
        density_high_nuclei_accepted = density_high_nuclei & accepted_mask
        density_low_nuclei_accepted = density_low_nuclei & accepted_mask
        

        # Percentage areas
        accepted_pixel_count = np.sum(accepted_mask)
        percentage_area_high_nuclei = (np.sum(density_high_nuclei_accepted) / accepted_pixel_count) * 100
        percentage_area_low_nuclei = (np.sum(density_low_nuclei_accepted) / accepted_pixel_count) * 100

        # Append the results for this image to the all_images_results DataFrame
        all_images_results = all_images_results.append({
            'Filename': filename,
            'Percentage_Area_High_nuclei': percentage_area_high_nuclei,
            'Percentage_Area_Low_nuclei': percentage_area_low_nuclei
        }, ignore_index=True)

        
        # Calculate and save nuclear packaging
        Local_Packing = weighted_kde_density_map(labels, num_points = 5000)
        Local_Packing = normalize_density_maps(Local_Packing)
        imwrite(os.path.join(output_folder, 'packaging', filename.replace('.tif', '_packaging.tif')),
               Local_Packing.astype(np.float32))

        # Create a Pandas DataFrame to store the region properties
        label_properties = regionprops_table(labels, intensity_image=img,
                                             properties=('label', 'area', 'eccentricity'))
        dataframe = pd.DataFrame(label_properties)

        num_clusters = 6

        # Calculate and save binned eccentricity
        eccentricity_binned_image = make_binned_image(labels, label_properties,'eccentricity', num_clusters)
        imwrite(os.path.join(output_folder, 'circularity', filename.replace('.tif', '_circularity.tif')),
               eccentricity_binned_image)

         # Calculate and save binned area
        area_binned_image = make_binned_image(labels, label_properties, 'area', num_clusters)
        imwrite(os.path.join(output_folder, 'area', filename.replace('.tif', '_area.tif')), area_binned_image)

        # Create and save mosaic
        make_mosaic_and_save(img, labels, num_clusters, Local_Density_mean_filter,
                             Local_Packing, eccentricity_binned_image, area_binned_image,
                             filename, output_folder)

        all_images_results.to_csv(os.path.join(output_folder, 'results', 
                                               'nuclei_density_groups_percentage_area.csv'), index=False)